In [1]:
import plotly.express as px
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, callback, Input, Output
import dash_daq as daq

In [2]:
population = pd.read_csv("../Dash_crossfiltering/population.csv")
new_books = pd.read_csv("../Dash_crossfiltering/new-books-per-million.csv")

In [3]:
data = pd.merge(new_books, population, how="inner", on=["Entity","Year"])
data = data[(data["Year"]>=1980) & (data["Year"]<=1996)]
countries_list = data[["Entity", "Year"]].groupby("Entity").count().sort_values(by="Year", ascending=False)
countries_list=countries_list[countries_list["Year"]==countries_list["Year"].max()]
data = pd.merge(data, countries_list, how="inner", on=["Entity"], suffixes=("","_z"))
data = data[data["Entity"]!="Russia"]

In [4]:
fig = px.scatter(data_frame=data,
                 x="Population (historical estimates)",
                 y="Book titles per capita (Fink-Jensen 2015)",
                 size="Population (historical estimates)",
                 animation_frame="Year",
                 animation_group="Entity",
                 text="Entity",
                 hover_name="Entity",
                 size_max=35,
                 range_x=[1,100000000], 
                 range_y=[1,5000],
                 template="plotly_white",
                 title="Population vs Book titles per capita in 1980-1996"
                 )
fig.update_traces(
    textposition="top center",
    marker=dict(
        line=dict(width=1, color="black"),  
    )
)
fig.update_yaxes(
    visible=True,
    showticklabels=True,
    title="# of Books",
    title_font=dict(size=14, color="black"),
    tickfont=dict(size=12, color="black")
)
fig.update_xaxes(
    visible=True,
    showticklabels=True,
    title="Population",
    title_font=dict(size=14, color="black"),
    tickfont=dict(size=12, color="black")
)
fig.update_xaxes(visible=True, showticklabels=True, title='Population')
fig.update_layout(
    width=1200, 
    height=600, 
    updatemenus=[
                 {
                     "type": "buttons",
                     "showactive": True,
                     "buttons": [{"label": "Start"}, {"label": "Stop"}]
                 }
    ]
)
fig.show()

In [5]:
def find_special_points(value, points):

    if value == points.max():
      special_point_name = "max"
    elif value == points.min():
      special_point_name = "min"
    elif value == points.median():
      special_point_name ="median"
    else: 
      special_point_name = "not special"

    return special_point_name

In [6]:
numbers = pd.DataFrame(np.random.randn(351), columns = ["sample",])
numbers["special_points"]=numbers["sample"].apply(lambda x: find_special_points(x,numbers["sample"]))

chart = px.scatter(
        numbers,
        x=numbers.index,
        y="sample",
        color="special_points",
        color_discrete_map={
            "max": "#800E13",
            "median": "#169873",
            "min": "#2C497F",
            "not special": "#E4DFDA",
                },
        render_mode="webgl",
        template='none',
        size=[10 for item in numbers["sample"]],
        title="Here're your points"
    )

cluster = [
           {
               "type": "circle",
               "x0": min(numbers.index), 
               "y0": min(numbers["sample"]+0.5),
               "x1": max(numbers.index)/2, 
               "y1": max(numbers["sample"]/2),
               "line": {"color": "DarkOrange", "width": 2, "dash": "dashdot"},
               "filcolor": "LightSkyBlue"
           }
]

chart.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(label="None",
                     method="relayout",
                     args=["shapes", []]),
                dict(label="Cluster",
                     method="relayout",
                     args=["shapes", cluster])
            ],
        )
    ]
)

In [7]:
app = Dash()

# App layout
app.layout = html.Div(children=[
    # A Boolean switch, toggles between ON and OFF
    daq.BooleanSwitch(id='bswitch', on=False),

    # A knob control, which allows selecting values by rotating it
    daq.Knob(id='knob'),

    # A div that displays information about the current state of the Boolean switch and the knob
    html.Div(id='bswitch-info'),

    # A gauge showing a value, controlled by the slider
    daq.Gauge(id='gauge', label="Gauge", value=8),

    # A graduated bar showing a value, controlled by the slider
    daq.GraduatedBar(id='gbar', label="Graduated bar", value=6),

    # A slider to select a value between 0 and 10
    html.Div(children=[
        dcc.Slider(
            id='slider',
            min=0, 
            max=10, 
            step=1, 
            value=3  # Default value
        )
    ], style={"width": "100%"}),
],
    # Styling for the main layout container
    style={
        "display": "flex",
        "flex-direction": "column",
        "justify-content": "center",
        "align-items": 'center',
        "align-content": "center",
        "height": "100%",
        "width": "75%",
        "fontFamily": "verdana",
        "color": "#444"
    }
)

# Callback to update the text in the 'bswitch-info' div
# Triggered by changes in the Boolean switch or knob value
@app.callback(
    Output('bswitch-info', 'children'),
    Input('bswitch', 'on'),  # Boolean switch state (True/False)
    Input('knob', 'value'),  # Current value of the knob
)
def update_bswitch_info(on, value):
    # Display the current state of the Boolean switch and the knob value
    return ('Switched {}.'.format(on) 
    + ', the knob is set to {}.'.format(value))

# Callback to update the values of the gauge and graduated bar
# Triggered by changes in the slider value
@app.callback(
    Output('gauge', 'value'),  # Update gauge value
    Output('gbar', 'value'),  # Update graduated bar value
    Input('slider', 'value'))  # Slider value as input
def update_gauge_and_bar(value):
    return value, value

# Callback to automatically toggle the Boolean switch based on the slider value
@app.callback(
    Output('bswitch', 'on'),  # Update Boolean switch state
    Input('slider', 'value'))  # Slider value as input
def update_bswitch_on_slider(value):
    # Set Boolean switch to True if slider value is 5 or more, otherwise False
    return value >= 5

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)